# Financial data science and hedging

#### Job Marcelis, Ernani Hazbolatow, Koen Verlaan

In [ ]:
from src.hedging_sim import *
import numpy as np
import matplotlib.pyplot as plt

#### 3. Hedging Simulation

The euler method is used to make a simple hedging simulation. The stock price movement is thus modeled according to the following equation:
\begin{equation}
    S_{t + dt} = S_t + rS_tdt + \sigma S_t\sqrt{dt}Z,
\end{equation}
where $r$ is the risk-free interest rate, $\sigma$ is the volatility, and $Z$ is a standard normal variable.

Furthermore, $\Delta$ is calculated using: $N(d_1)$, where $N$ is the cumulative distribution function of the standard normal distribution and $d_1$ is given by:
\begin{equation}
    d_1 = \frac{ln(\frac{S}{K}) + (r + \frac{\sigma ^2}{2})T}{\sigma \sqrt{T}},
\end{equation}
where $K$ is the strike price and $T$ is the time. And the option price is given by:
\begin{equation}
    C(S_t, K, t, T, \sigma) = S_tN(d_1) - Ke^{-rT}N(d_2),
\end{equation}
where $d_2 = d_1 - \sigma \sqrt{T}$.

The hedging simulation works as follows:
- A price path is calculated using the euler approximation of the Black-Scholes equation
- At each step in this path, the $\Delta$ calculated, the interest rate is added, and the change in stock position is calculated
- At maturity, the payoff of the option, followed by the value of the portfolio, and the hedging error, is calculated.

This process is repeated 5000 times and the mean and confidence intervals at the $p=95%$ confidence level are calculated.

In the first experiment, we match the volatility of the price path with the option delta and vary the hedge adjustment frequency:

In [ ]:
more_freqs = np.linspace(12, 330, 11)
means = []
CIs = []
for i in more_freqs:
    mean, CI = multiple_hedge_sims(num_runs=5000, S0=100, r=0.06, sigma=0.2, sigma_delta=0.2, T=1, dt=1/i, K=99, seed=int(i))
    means.append(mean)
    CIs.append(CI)

mean_weekly, CI_weekly = multiple_hedge_sims(num_runs=5000, S0=100, r=0.06, sigma=0.2, sigma_delta=0.2, T=1, dt=1/52, K=99, seed=52)
mean_daily, CI_daily = multiple_hedge_sims(num_runs=5000, S0=100, r=0.06, sigma=0.2, sigma_delta=0.2, T=1, dt=1/365, K=99, seed=365)

In [ ]:
plt.figure(figsize=(8,5), dpi=300)
plt.title("Hedging error vs. Frequency of Hedge Adjustments", fontsize=16)
plt.errorbar(more_freqs, means, yerr=CIs, fmt='o', color='black', label='Other Frequencies', capsize=5)
plt.errorbar(52, mean_weekly, yerr=CI_weekly, fmt='o', color='red', label='Weekly Frequency', capsize=5)
plt.errorbar(365, mean_daily, yerr=CI_daily, fmt='o', color='green', label='Daily Frequency', capsize=5)
plt.ylabel("Hedging error [EUR]", fontsize=15)
plt.xlabel("Frequency of Hedge Adjustments per Year", fontsize=15)
plt.tick_params(axis='both', labelsize=12)
plt.legend(fontsize=12)
plt.grid(ls='dashed')
plt.show()

In the second experiment, the volatility of the price path is fixed while the volatility of the option delta is varied:

In [ ]:
delta_val_volatility = np.linspace(10, 30, 10)
means = []
CIs = []
for i in delta_val_volatility:
    mean, CI = multiple_hedge_sims(num_runs=5000, S0=100, r=0.06, sigma=0.2, sigma_delta=i/100, T=1, dt=1/365, K=99, seed=int(i*i))
    means.append(mean)
    CIs.append(CI)
mean_matched, CI_matched = multiple_hedge_sims(num_runs=5000, S0=100, r=0.06, sigma=0.2, sigma_delta=20/100, T=1, dt=1/365, K=99, seed=69)

In [ ]:
plt.figure(figsize=(8,5), dpi=300)
plt.title("Hedging error vs. Volatility of Delta Valuation", fontsize=16)
plt.errorbar(delta_val_volatility, means, yerr=CIs, fmt='o', color='black', capsize=5, label='Mismatched Volatility')
plt.errorbar(20, mean_matched, yerr=CI_matched, fmt='o', color='green', capsize=5, label='Matched Volatility')
plt.ylabel("Hedging error [EUR]", fontsize=15)
plt.xlabel("Volatility of Delta Valuation [%]", fontsize=15)
plt.tick_params(axis='both', labelsize=12)
plt.grid(ls='dashed')
plt.legend(fontsize=12)
plt.show()